In [1]:
from datetime import datetime
import pandas_datareader.data as web
import numpy as np

In [ ]:
forex_df = web.DataReader(name="GBP/INR", data_source="av-forex-daily", start=datetime(2014,1,1), end=datetime(2019,12,31), api_key="7Y3UNPFOBVKM8XPA7Y3UNPFOBVKM8XPA")

In [ ]:
forex_open_df = forex_df["open"]

In [ ]:
forex_series = forex_open_df.to_numpy()

In [ ]:
def split_seq(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence) - 1:
            break
        seqx = sequence[i:end_ix]
        X.append(seqx)
    return np.array(X)



In [ ]:
n_steps = 50
series = split_seq(forex_series, n_steps + 10)
series.shape

In [ ]:
series = series.reshape((series.shape[0], n_steps + 10, 1))
series.shape

In [ ]:
# use these values tp normalize the data
# and then renormalize your predictions

mean = np.mean(series)
std = np.std(series)
max = np.max(series)
min = np.min(series)

print(f"mean = {mean}, std = {std}, max = {max}, min = {min}")

In [ ]:
series = (series - mean) / std 
series.shape

In [ ]:
X_train, X_test = series[:1200, :n_steps], series[1200:-1, :n_steps]
print(f"{X_train.shape}, {X_test.shape}")

In [ ]:
from numpy import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2

SEED = 32
random.random(SEED)
tf.random.set_seed(SEED)

In [ ]:
Y = np.empty((1282, n_steps, 10))

for step_ahead in range(1,10 + 1):
    Y[:,:, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]

Y_train = Y[:1200]
Y_test = Y[1200:-1]

In [ ]:
print(f"{Y_train.shape}, {Y_test.shape}")

In [ ]:
keras.backend.clear_session()

model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None,1]))
for rate in (1,2,4,8) * 2:
    model.add(keras.layers.Conv1D(
        filters=20, kernel_size=2, padding="causal",
        activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1, kernel_regularizer=l2(0.2)))

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.001)

model.compile(loss="mse", optimizer=optimizer)
history = model.fit(X_train, Y_train, epochs=300, validation_split=0.2, verbose=0, shuffle=False)

In [ ]:
mse_test = model.evaluate(X_test, Y_test)

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 

pd.DataFrame(history.history).plot(figsize=(8,5))
plt.grid(True)
axes = plt.gca()
axes.set_ylim(0,1)
plt.show()

In [ ]:
X_new = X_test[0]
X_new = X_new.reshape((1,50,1))
X_new.shape

In [ ]:
y_hat = model.predict(X_new)

In [ ]:
X_prime = np.load("X_prime.npy")
X_prime.shape

In [ ]:
X_prime_norm = (X_prime - mean) / std 

In [ ]:
y_prime = model.predict(X_prime_norm)

In [ ]:
pred_y_norm = y_prime[0]
pred_y_norm

In [ ]:
pred_y_renorm = (pred_y_norm * std) + mean
pred_y_renorm

In [ ]:
pred_y_renorm[49]

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(y_hat[0][49], label="prediction")
plt.plot(Y_test[0][49], label="actual")
plt.legend()
plt.show()

In [ ]:

X_norm = (X_test * std) + mean
X_norm.shape

In [ ]:
model.save("mvp1a_mvp.h5")

### calculate rate of chnage of actual and predicted prices (subset)


In [ ]:
#change rate
actual = np.array([0.0824, 0.607, 0.6238, 0.2756, 0.2074]) 
prediction = np.array([0.39015, 0.185175, -0.236245, -0.437934, 0.263899])


In [ ]:
plt.figure(figsize=(8,5))
plt.title("Rate of change of actual and predicted prices")
plt.plot(actual, label="actual")
plt.plot(prediction, label="prediction")
plt.legend()
plt.show()

### test prediction for next 10 events

In [ ]:
prices_past = np.load("prices_past.npy")

In [ ]:
prices_past.shape

In [ ]:
prices_past

In [ ]:
# normalise the data in prices_past
prices_past_norm = (prices_past - mean) / std
prices_past_norm

In [ ]:
new_pred = model.predict(prices_past_norm)

In [ ]:
new_pred.shape

In [ ]:
# renormalise the data
new_pred_renorm = (new_pred * std) + mean
new_pred_renorm.shape

In [ ]:
ave_next_50X10_day_prices = np.average(new_pred_renorm)
np.save("ave_next_50X10_day_prices", ave_next_50X10_day_prices)

In [ ]:
item_1_10_day_prediction = new_pred_renorm[0][0]
np.save("item_1_of_10_day_prediction", item_1_10_day_prediction)

In [ ]:
item_50_10_day_prediction = new_pred_renorm[0][49]
np.save("item_50_of_10_day_prediction", item_50_10_day_prediction)

In [ ]:
sorted_period_prices = []
for i in range(50):
    sorted_daily_prices = []
    for j in range(10):
        sorted_daily_prices.append(new_pred_renorm[0][i][j])
    sorted_period_prices.append(sorted_daily_prices)



In [ ]:
sorted_period_prices = np.array(sorted_period_prices)
np.save("sorted_period_prices", sorted_period_prices)

In [ ]:
period1_prices = sorted_period_prices[0]
print(f"average of period1 prices = {np.average(period1_prices)}")
print(f"max of period1 prices = {np.max(period1_prices)}")
print(f"min of period1 prices = {np.min(period1_prices)}")
print(f"std of period1 prices = {np.std(period1_prices)}")

In [ ]:
period2_prices = sorted_period_prices[1]
print(f"average of period2 prices = {np.average(period2_prices)}")
print(f"max of period2 prices = {np.max(period2_prices)}")
print(f"min of period2 prices = {np.min(period2_prices)}")
print(f"std of period2 prices = {np.std(period2_prices)}")

In [ ]:
period3_prices = sorted_period_prices[2]
print(f"average of period3 prices = {np.average(period3_prices)}")
print(f"max of period3 prices = {np.max(period3_prices)}")
print(f"min of period3 prices = {np.min(period3_prices)}")
print(f"std of period3 prices = {np.std(period3_prices)}")